In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("dados_completos_anotados.csv")
df.shape

(400, 9)

In [3]:
df.head()

,Unnamed: 0,link,titulo,citacoes,ano,tipo_arquivo,versao,busca,y
0,0,https://www.academia.edu/download/55091016/alg...,Introducción a los algoritmos genéticos y la p...,['75'],['2010'],[LIVRO],['3'],algoritmos+geneticos,1
1,1,https://www.researchgate.net/profile/Pablo_Est...,Optimización mediante algoritmos genéticos,['51'],['1997'],[PDF],['2'],algoritmos+geneticos,0
2,2,http://it.uc3m.es/jvillena/irc/practicas/06-07...,Algoritmos genéticos,['18'],"['2007', '3']",[PDF],['5'],algoritmos+geneticos,0
3,3,http://cursos.itam.mx/akuri/PUBLICA.CNS/2000/A...,Algoritmos genéticos,['26'],['2002'],[LIVRO],['2'],algoritmos+geneticos,0
4,4,https://the-geek.org/docs/algen/algen.html,Algoritmos genéticos y computación evolutiva,['29'],['2004'],[HTML],['2'],algoritmos+geneticos,0


### Limpeza e transformação

In [4]:
df_limpo = df.copy()

In [5]:
df_limpo = df_limpo.drop(['Unnamed: 0'], axis=1)
df_limpo.head(1)

,link,titulo,citacoes,ano,tipo_arquivo,versao,busca,y
0,https://www.academia.edu/download/55091016/alg...,Introducción a los algoritmos genéticos y la p...,['75'],['2010'],[LIVRO],['3'],algoritmos+geneticos,1


In [6]:
def clean_data(cols_list, char_list):
    for col in cols_list:
        for char in char_list:
            df_limpo[col] = df_limpo[col].str.replace(char, '')

In [7]:
cols_to_clean = ['citacoes', 'ano', 'tipo_arquivo', 'versao']
spec_chars = ["[", "]", "'", " "]

clean_data(cols_to_clean, spec_chars)
df_limpo.head()

,link,titulo,citacoes,ano,tipo_arquivo,versao,busca,y
0,https://www.academia.edu/download/55091016/alg...,Introducción a los algoritmos genéticos y la p...,75,2010,LIVRO,3,algoritmos+geneticos,1
1,https://www.researchgate.net/profile/Pablo_Est...,Optimización mediante algoritmos genéticos,51,1997,PDF,2,algoritmos+geneticos,0
2,http://it.uc3m.es/jvillena/irc/practicas/06-07...,Algoritmos genéticos,18,"2007,3",PDF,5,algoritmos+geneticos,0
3,http://cursos.itam.mx/akuri/PUBLICA.CNS/2000/A...,Algoritmos genéticos,26,2002,LIVRO,2,algoritmos+geneticos,0
4,https://the-geek.org/docs/algen/algen.html,Algoritmos genéticos y computación evolutiva,29,2004,HTML,2,algoritmos+geneticos,0


In [8]:
cols_to_convert = ['citacoes', 'versao']
df_limpo[cols_to_convert] = df_limpo[cols_to_convert].astype('int32')
df_limpo.dtypes

link            object
titulo          object
citacoes         int32
ano             object
tipo_arquivo    object
versao           int32
busca           object
y                int64
dtype: object

#### Limpando coluna 'ano'

In [9]:
for i in df_limpo['ano']:
    if(len(i)>4):
        print(i)

2007,3
2010,200,62,146,19
2011,2011,40
85,2002
2009,150,214,190,154
2014,2014
2010,3
94,1994
4,2011
02,2002
2010,2010
2006,2006
2014,47,2014
8,2003
2008,1
1611,01236,2016
2000,2000
40,2002
1708,07747,2017
33,2010
2001,2001
2010,2010
2017,2017
10,939,288,2005
25,2016
2011,27,2011
1307,0411,2013
1206,4656,2012
2005,2005
1702,08608,2017
21,2012
1605,07277,2016


In [10]:
coluna_ano = df_limpo['ano'].copy()

In [11]:
import re

for i in range(len(coluna_ano)):
    print(i)
    if(len(coluna_ano[i])>4):
        res = re.findall(r'\d{4}', coluna_ano[i])
        if(len(res)>1):
            coluna_ano[i] = res[len(res)-1]
        else:
            coluna_ano[i] = res[0]
print(coluna_ano)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [12]:
df_limpo['ano'] = coluna_ano
df_limpo.head()

,link,titulo,citacoes,ano,tipo_arquivo,versao,busca,y
0,https://www.academia.edu/download/55091016/alg...,Introducción a los algoritmos genéticos y la p...,75,2010,LIVRO,3,algoritmos+geneticos,1
1,https://www.researchgate.net/profile/Pablo_Est...,Optimización mediante algoritmos genéticos,51,1997,PDF,2,algoritmos+geneticos,0
2,http://it.uc3m.es/jvillena/irc/practicas/06-07...,Algoritmos genéticos,18,2007,PDF,5,algoritmos+geneticos,0
3,http://cursos.itam.mx/akuri/PUBLICA.CNS/2000/A...,Algoritmos genéticos,26,2002,LIVRO,2,algoritmos+geneticos,0
4,https://the-geek.org/docs/algen/algen.html,Algoritmos genéticos y computación evolutiva,29,2004,HTML,2,algoritmos+geneticos,0


In [13]:
df_limpo['ano'] = df_limpo['ano'].astype('int32')

In [14]:
from sklearn.preprocessing import OneHotEncoder
encoded_column = df_limpo['tipo_arquivo']
## Precisamos transformar a coluna em um numpy array para pode transformá-la
encoded_column = encoded_column.to_numpy()
type(encoded_column)

numpy.ndarray

In [15]:
encoder = OneHotEncoder(handle_unknown='ignore')

enc_columns = pd.DataFrame(encoder.fit_transform(encoded_column.reshape(-1,1)).toarray())
enc_columns.head()

,0,1,2,3,4
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0


In [16]:
new_labels = ['CITACAO', 'DOC', 'HTML', 'LIVRO', 'PDF']
enc_columns.columns = new_labels
enc_columns.head(1)

,CITACAO,DOC,HTML,LIVRO,PDF
0,0.0,0.0,0.0,1.0,0.0


In [17]:
df_limpo = df_limpo.join(enc_columns)
df_limpo.head(1)

,link,titulo,citacoes,ano,tipo_arquivo,versao,busca,y,CITACAO,DOC,HTML,LIVRO,PDF
0,https://www.academia.edu/download/55091016/alg...,Introducción a los algoritmos genéticos y la p...,75,2010,LIVRO,3,algoritmos+geneticos,1,0.0,0.0,0.0,1.0,0.0


In [18]:
df_limpo = df_limpo.drop(['tipo_arquivo'], axis=1)
df_limpo.head(1)

,link,titulo,citacoes,ano,versao,busca,y,CITACAO,DOC,HTML,LIVRO,PDF
0,https://www.academia.edu/download/55091016/alg...,Introducción a los algoritmos genéticos y la p...,75,2010,3,algoritmos+geneticos,1,0.0,0.0,0.0,1.0,0.0


#### Transformando a coluna 'titulo'

In [19]:
coluna_titulo = df_limpo['titulo']
coluna_titulo

0      Introducción a los algoritmos genéticos y la p...
1             Optimización mediante algoritmos genéticos
2                                   Algoritmos genéticos
3                                   Algoritmos genéticos
4           Algoritmos genéticos y computación evolutiva
                             ...                        
395    Machine learning for molecular and materials s...
396    Speeding up distributed machine learning using...
397                     Multi-objective machine learning
398    Transferability in machine learning: from phen...
399    Using machine learning to break visual human i...
Name: titulo, Length: 400, dtype: object

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1,3))
titulo_vec = vectorizer.fit_transform(coluna_titulo)

In [21]:
titulo_vec.shape

(400, 972)

In [22]:
df_titulo = pd.DataFrame(titulo_vec.toarray(), columns=vectorizer.get_feature_names())
df_titulo.head(1)

,abastecimento,abastecimiento,abastecimiento de,abastecimiento de agua,abordagem,abordagem de,abordagem de algoritmos,adaptativo,adversarial,adversarial machine,...,água,índices,índices bursátiles,óptima,óptima de,óptimo,óptimo de,ótima,ótima de,ótima de sistemas
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
df_limpo.rename(columns={'busca': 'busca_executada'}, inplace=True)

In [24]:
df_limpo.head(1)

,link,titulo,citacoes,ano,versao,busca_executada,y,CITACAO,DOC,HTML,LIVRO,PDF
0,https://www.academia.edu/download/55091016/alg...,Introducción a los algoritmos genéticos y la p...,75,2010,3,algoritmos+geneticos,1,0.0,0.0,0.0,1.0,0.0


In [25]:
df_limpo = df_limpo.join(df_titulo)
df_limpo.head(1)

,link,titulo,citacoes,ano,versao,busca_executada,y,CITACAO,DOC,HTML,...,água,índices,índices bursátiles,óptima,óptima de,óptimo,óptimo de,ótima,ótima de,ótima de sistemas
0,https://www.academia.edu/download/55091016/alg...,Introducción a los algoritmos genéticos y la p...,75,2010,3,algoritmos+geneticos,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
y = df_limpo.y
X = df_limpo.drop(['y', 'titulo', 'link', 'busca_executada'], axis=1)

In [27]:
X.shape

(400, 980)

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [29]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, random_state=0, min_samples_leaf=1, class_weight='balanced', n_jobs=2)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=2, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [30]:
predicted = rf.predict_proba(X_test)[:, 1]
predicted

array([0.052, 0.318, 0.956, 0.038, 0.841, 0.071, 0.178, 0.918, 0.083,
       0.859, 0.066, 0.051, 0.221, 0.734, 0.67 , 0.042, 0.754, 0.073,
       0.632, 0.062, 0.023, 0.756, 0.003, 0.654, 0.078, 0.93 , 0.119,
       0.063, 0.916, 0.112, 0.033, 0.875, 0.829, 0.978, 0.013, 0.228,
       0.045, 0.033, 0.032, 0.211, 0.16 , 0.813, 0.381, 0.276, 0.441,
       0.039, 0.728, 0.388, 0.049, 0.988, 0.923, 0.014, 0.973, 0.106,
       0.701, 0.662, 0.013, 0.077, 0.014, 0.053, 0.033, 0.889, 0.147,
       0.812, 0.087, 0.66 , 0.047, 0.061, 0.135, 0.682, 0.101, 0.393,
       0.007, 0.182, 0.032, 0.082, 0.889, 0.614, 0.913, 0.787])

In [31]:
from sklearn.metrics import roc_auc_score, average_precision_score
roc_auc = roc_auc_score(y_test, predicted)
ap = average_precision_score(y_test, predicted)

print("AUC: {} | Average Precision: {}".format(roc_auc, ap))

AUC: 0.9117445054945056 | Average Precision: 0.8975543010873304


## Parâmetros testados do Random Forest

### n_estimators=100, min_samples_leaf=1, min_df=2:
     AUC: 0.9385 | AP: 0.9328

### n_estimators=100, min_samples_leaf=2, min_df=2:
     AUC: 0.9244 | AP: 0.8784

### n_estimators=1000, min_samples_leaf=1, min_df=2:
     AUC: 0.9375 | AP: 0.9300

## Parâmetros testados do TfidfVectorizer

### min_df=2, n_gram(1,1):
    AUC: 0.9375 | AP: 0.9300

### min_df=2, n_gram(1,2):
    AUC: 0.9265 | AP: 0.9213

### min_df=2, n_gram(1,3): 
    AUC: 0.9117 | AP: 0.8975
    

## LightGBM

In [32]:
from lightgbm import LGBMClassifier

In [33]:
lgbm = LGBMClassifier(random_state=0, class_weight="balanced", n_jobs=2)
lgbm.fit(X_train, y_train)

LightGBMError: Do not support non-ASCII characters in feature name.

In [34]:
df_limpo.ano = df_limpo.ano.astype('int32')

In [35]:
X_train.dtypes

citacoes               int32
ano                    int32
versao                 int32
CITACAO              float64
DOC                  float64
                      ...   
óptimo               float64
óptimo de            float64
ótima                float64
ótima de             float64
ótima de sistemas    float64
Length: 980, dtype: object

In [36]:
lgbm.fit(X_train, y_train)

LightGBMError: Do not support non-ASCII characters in feature name.

#### Precisamos remover a acentuação para usar o LightGBM

In [37]:
df_lgbm = df_limpo.copy()
df_lgbm.head(1)

,link,titulo,citacoes,ano,versao,busca_executada,y,CITACAO,DOC,HTML,...,água,índices,índices bursátiles,óptima,óptima de,óptimo,óptimo de,ótima,ótima de,ótima de sistemas
0,https://www.academia.edu/download/55091016/alg...,Introducción a los algoritmos genéticos y la p...,75,2010,3,algoritmos+geneticos,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
import unidecode
df_lgbm.columns = [unidecode.unidecode(x) for x in df_lgbm.columns]
df_lgbm.columns

Index(['link', 'titulo', 'citacoes', 'ano', 'versao', 'busca_executada', 'y',
       'CITACAO', 'DOC', 'HTML',
       ...
       'agua', 'indices', 'indices bursatiles', 'optima', 'optima de',
       'optimo', 'optimo de', 'otima', 'otima de', 'otima de sistemas'],
      dtype='object', length=984)

In [39]:
df_lgbm.head(1)

,link,titulo,citacoes,ano,versao,busca_executada,y,CITACAO,DOC,HTML,...,agua,indices,indices bursatiles,optima,optima de,optimo,optimo de,otima,otima de,otima de sistemas
0,https://www.academia.edu/download/55091016/alg...,Introducción a los algoritmos genéticos y la p...,75,2010,3,algoritmos+geneticos,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
y_lgbm = df_lgbm.y
X_lgbm = df_lgbm.drop(['y', 'titulo', 'link', 'busca_executada'], axis=1)

In [41]:
X_train_lgbm, X_test_lgbm, y_train_lgbm, y_test_lgbm = train_test_split(X_lgbm, y_lgbm,
                                                                                        test_size=0.2, random_state=0)
lgbm.fit(X_train_lgbm, y_train_lgbm)

LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_jobs=2, num_leaves=31,
               objective=None, random_state=0, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [42]:
predict = lgbm.predict_proba(X_test_lgbm)[:, 1]
predict

array([5.12640212e-04, 1.32576451e-01, 9.97637509e-01, 7.71439875e-04,
       8.77785869e-01, 5.29382114e-03, 8.84935619e-02, 9.96012198e-01,
       4.51527511e-04, 9.92612126e-01, 3.12628500e-02, 1.64777105e-02,
       2.45783915e-02, 8.59009466e-01, 9.21112294e-01, 2.05347337e-03,
       9.99866283e-01, 5.68825194e-03, 2.37119494e-01, 1.26041123e-03,
       3.32374141e-04, 1.90441237e-01, 8.74360552e-04, 6.12327781e-01,
       1.26840089e-03, 9.97734798e-01, 5.04216923e-03, 2.91750144e-01,
       7.95529018e-01, 3.72548323e-03, 1.95949574e-03, 9.95660698e-01,
       9.31416625e-01, 9.98929390e-01, 2.45514451e-03, 2.28660600e-04,
       1.78435606e-03, 1.10250377e-03, 4.94902584e-03, 6.48327772e-02,
       9.35927870e-02, 9.99770117e-01, 8.21775516e-01, 5.82630563e-01,
       5.61888720e-01, 8.49683865e-04, 8.85941149e-01, 7.70418896e-01,
       2.91971797e-03, 9.99789150e-01, 9.79604702e-01, 5.39385256e-04,
       9.99606679e-01, 1.74236865e-03, 9.04460601e-01, 1.38389052e-01,
      

In [43]:
auc_lgbm = roc_auc_score(y_test_lgbm, predict)
auc_lgbm

0.9333791208791208

In [44]:
ap = average_precision_score(y_test_lgbm, predict)
ap

0.9058952339340935

#### Otimizando os parâmetros do LightGBM

In [48]:
##Função de minimização usada para otimizar os parâmetros
from skopt import forest_minimize

In [50]:
def tune_lgbm(params):
    lr = params[0] ##learning rate
    max_depth = params[1] ##profundidade da árvore
    min_child_samples = params[2] ##número mínimo de exemplos em cada nó
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    
    tuned_lgbm = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         min_child_samples=min_child_samples, subsample=subsample,
                         colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                         class_weight="balanced", n_jobs=2)
    tuned_lgbm.fit(X_train_lgbm, y_train_lgbm)
    p = tuned_lgbm.predict_proba(X_test_lgbm)[:, 1]
    
    print(roc_auc_score(y_test_lgbm, p))
    
    ## É retornado a negativa da métrica pois usamos uma função de minimização
    return -average_precision_score(y_test_lgbm, p)

param_values = [(1e-3, 1e-1, 'log-uniform'), ##learning rate
               (1, 10), ##max_depth
               (1, 20), ##min_child_sample
               (0.05, 1.), ##sample
               (0.05, 1.), ##colsample_bytree
               (100, 1000)] ##n_estimators

result = forest_minimize(tune_lgbm, param_values, random_state=160745, n_random_starts=20, n_calls=50, verbose=1)

Iteration No: 1 started. Evaluating function at random point.
0.9203296703296703
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 1.2500
Function value obtained: -0.8993
Current minimum: -0.8993
Iteration No: 2 started. Evaluating function at random point.
0.9223901098901098
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.3320
Function value obtained: -0.8925
Current minimum: -0.8993
Iteration No: 3 started. Evaluating function at random point.
0.9292582417582418
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.4100
Function value obtained: -0.9080
Current minimum: -0.9080
Iteration No: 4 started. Evaluating function at random point.
0.9375
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 2.2920
Function value obtained: -0.9104
Current minimum: -0.9104
Iteration No: 5 started. Evaluating function at random point.
0.9539835164835165
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.5800


0.9436813186813187
Iteration No: 38 ended. Search finished for the next optimal point.
Time taken: 0.7900
Function value obtained: -0.9135
Current minimum: -0.9426
Iteration No: 39 started. Searching for the next optimal point.
0.9429945054945055
Iteration No: 39 ended. Search finished for the next optimal point.
Time taken: 0.6100
Function value obtained: -0.9097
Current minimum: -0.9426
Iteration No: 40 started. Searching for the next optimal point.
0.9416208791208791
Iteration No: 40 ended. Search finished for the next optimal point.
Time taken: 0.5300
Function value obtained: -0.9155
Current minimum: -0.9426
Iteration No: 41 started. Searching for the next optimal point.
0.9375
Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.5300
Function value obtained: -0.9077
Current minimum: -0.9426
Iteration No: 42 started. Searching for the next optimal point.
0.9072802197802197
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0

#### Melhores parâmetros encontrados

In [51]:
result.x

[0.05692306574259571, 2, 14, 0.7491505567388204, 0.9530435710977047, 201]

#### AUC: 0.9581043956043955
#### AP: 0.9426

## Logistic Regression

In [52]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)

c:\users\alexandre\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
p_lr = lr.predict_proba(X_test)[:, 1]
p_lr

array([0.25762853, 0.54672799, 0.83338536, 0.07724336, 0.79994359,
       0.21628447, 0.17146125, 0.72137697, 0.1115592 , 0.82969607,
       0.10124857, 0.12624101, 0.30252594, 0.85649195, 0.17205663,
       0.10947238, 0.61920424, 0.15732481, 0.80121529, 0.06962906,
       0.08127619, 0.63471869, 0.08330976, 0.73250023, 0.08650765,
       0.81416089, 0.2191475 , 0.22003281, 0.65327655, 0.11744174,
       0.26138513, 0.76836362, 0.44129447, 0.99302181, 0.07977986,
       0.0933418 , 0.05751861, 0.11699123, 0.07044449, 0.41622952,
       0.25381445, 0.7987819 , 0.24788106, 0.08929002, 0.38148292,
       0.07099911, 0.32830511, 0.44172204, 0.07218968, 0.78220093,
       0.68907053, 0.06930212, 0.80044026, 0.26910589, 0.45907731,
       0.8258995 , 0.11709013, 0.13221079, 0.11260739, 0.12680119,
       0.08911859, 0.69984627, 0.21830086, 0.62545447, 0.08987978,
       0.455777  , 0.06754093, 0.33582636, 0.37313821, 0.73503702,
       0.1204968 , 0.54435534, 0.0774435 , 0.10760344, 0.22455

In [55]:
roc_auc_lr = roc_auc_score(y_test, p_lr)
ap_lr = average_precision_score(y_test, p_lr)

print("AUC: {} | Average Precision: {}".format(roc_auc_lr, ap_lr))

AUC: 0.8990384615384616 | Average Precision: 0.7938174206214182


In [56]:
import joblib as jb

## Salvando os modelos

In [57]:
jb.dump(vectorizer, "vectorizer.pkl.z")
jb.dump(rf, "random_forest.pkl.z")
jb.dump(lgbm, "lgbm.pkl.z")

['lgbm.pkl.z']